In [46]:
with open("../data/day20.txt") as f:
    data = f.read()
import numpy as np

arr = np.array([list(row) for row in data.split("\n")])

In [47]:
directions = [
    np.array((-1, 0)),
    np.array((0, -1)),
    np.array((1, 0)),
    np.array((0, 1)),
]


def is_outside_bounds(test_position, size):
    return (
        test_position[0] == -1
        or test_position[1] == -1
        or test_position[0] == size[0]
        or test_position[1] == size[1]
    )


def find_possible_directions(arr, current_position):
    possible_directions = []
    for possible_step in directions:
        possible_next_position = tuple(current_position + possible_step)
        if arr[possible_next_position] != "#":
            possible_directions.append(possible_step)
    return possible_directions


temp_arr = arr.copy()
starting_position = np.argwhere(temp_arr == "S")[0]
starting_direction = np.array([0, 1])

current_position = tuple(starting_position)
current_route = [current_position]
routes_in_consideration = [current_route]
all_routes = []

while True:
    try:
        current_route = routes_in_consideration.pop(0)
    except IndexError:
        break
    current_position = current_route[-1]
    possible_directions = find_possible_directions(temp_arr, current_position)
    for step in possible_directions:
        new_position = tuple(current_position + step)
        if new_position in current_route:
            continue  # Do not walk in loops
        new_route = current_route + [new_position]
        if temp_arr[new_position] == "E":
            all_routes.append(new_route)
        else:
            routes_in_consideration.append(new_route)

In [48]:
route = all_routes[0]

In [28]:
def find_possible_cheats(position, route, grid):
    possible_cheats = []
    for direction in directions:
        index = route.index(position)
        cheat_available = (
            tuple(position + 2 * direction) in route[index + 1 :]
        )  # we need to cheat forward :)
        cheat_through_wall = (
            grid[tuple(position + direction)] == "#"
        )  # Cheat thorugh a wall, don't go through normal route
        if cheat_available and cheat_through_wall:
            saved_seconds = route.index(tuple(position + 2 * direction)) - index - 2
            possible_cheats.append(saved_seconds)
    return possible_cheats

In [29]:
saved_seconds = []
for position in route[:-2]:
    saved_seconds.extend(find_possible_cheats(position, route, temp_arr))

In [34]:
len([cheat for cheat in saved_seconds if cheat >= 100])

1502

# Part 2

In [49]:
import itertools

In [79]:
max_time_cheat = 20
min_saved_time = 100

In [80]:
time_taken = {}

for time in range(max_time_cheat, 1, -1):
    combinations = itertools.combinations_with_replacement(directions, time)
    for comb in combinations:
        total_distance = tuple(sum(comb))
        time_taken[total_distance] = time

del time_taken[(0, 0)]  # This can never save time

In [70]:
position_index = {position: i for i, position in enumerate(route)}

In [75]:
def find_possible_cheats(position, route, cheats):
    possible_cheats = {}
    start_index = position_index[position]
    for cheat, cheat_time in cheats.items():
        final_position = tuple(position + np.array(cheat))
        try:
            final_index = position_index[final_position]
        except KeyError:  # end on wall :(
            continue
        time_normal_route = final_index - start_index
        time_saved = time_normal_route - cheat_time

        possible_cheats[(position, final_position)] = time_saved
    return possible_cheats

In [76]:
import tqdm

In [78]:
saved_seconds = []
for position in tqdm.tqdm(route[:-min_saved_time]):
    saved_seconds.append(find_possible_cheats(position, route, time_taken))

  0%|          | 0/9357 [00:00<?, ?it/s]

100%|██████████| 9357/9357 [00:47<00:00, 194.96it/s]


In [81]:
result = {}
for d in saved_seconds:
    result.update(d)
result = {k: v for k, v in result.items() if v >= min_saved_time}

In [82]:
len(result)

1028136